<a href="https://colab.research.google.com/github/garylau1/vertex-minors/blob/master/Complete_script_customerData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## In this task we would do a vision classification task in Brain tumor from scatch.

### Our result:
The accuracy is around 75 in testing set after trying different layers.
This model is very well in detecting class 1,2 and 3 (meningioma_tumor,no_tumor nd pituitary_tumour) but fail to detect class 0 (glimoa_tumor).

### The most significance is:

- We build up our linear layers ,relu layers and loss functions from scatch from basic class objects(e.g writing the whole formula inside the softmax function and cross-entropy loss)


- We build up out custom dataset from scatch as well.


- We write into python scripts and run it directly using command line in colab.

### Future work

- We might try transfer learning
- We can try different paraemeters



In [1]:
%%writefile path_loader.py

from pathlib import Path
import os

out_path=Path("out_model_set")
os.makedirs(out_path,exist_ok=True)



Imagepath=("/content/Brain-Tumor-Classification-DataSet")

Writing path_loader.py


In [2]:
!python path_loader.py

In [3]:
%%writefile out_model_set/data_creator.py

import requests
import zipfile
from pathlib import Path
import pathlib
from PIL import Image
import random
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms

from torch.utils.data import Dataset, DataLoader
import os





def find_classes(dict1):
  """find the class fikder bane"""

  found=sorted([entry1.name for entry1 in list(os.scandir(dict1))if entry1.is_dir()])
  if not found:
      raise  FileNotFoundError(f"no such class1111 in {dict1}")

  class_index={class_name: i for i,class_name in enumerate(found)}
  return found,class_index




class customdataset(Dataset):

  def find_classes(self,dict1):

    found=sorted([entry1.name for entry1 in list(os.scandir(dict1))if entry1.is_dir()])
    #found=[entry1.name for entry1 in list(os.scandir(dict1))if entry1.is_dir()]
    if not found:
      raise  FileNotFoundError(f"no such class1111 in {dict1}")

    class_index={class_name: i for i,class_name in enumerate(found)}
    return found,class_index


  def __init__(self,target_folder,transform=None):
    #self.targetpath=list(target_folder.glob("*/*.jpg"))
    self.targetpath=sorted(list(pathlib.Path(target_folder).glob("*/*.jpg")))
    self.transform=transform
    self.classes,self.class_to_idx=self.find_classes(target_folder)

    self.transform_2=transforms.Compose(
    [transforms.Resize((224,224)),transforms.RandomHorizontalFlip(p=0.5),transforms.ToTensor()])


  def load_image(self,index):
      return (Image.open(self.targetpath[index]))
  def __len__(self):
      a=self.targetpath
      return len(a)

  def __getitem__(self,inx):

    load=(self.load_image(inx))
    name=self.targetpath[inx].parent.name
    class_index=self.class_to_idx[name]


    if self.transform:
      return self.transform(load),class_index
    else:
      return self.transform_2(load),class_index


data_transform_aug=transforms.Compose([transforms.Resize((228,228)),transforms.TrivialAugmentWide(num_magnitude_bins=31),transforms.ToTensor()])

data_transform_simple=transforms.Compose(
    [transforms.Resize((228,228)),transforms.ToTensor()])

def customdataset_loader(batchs,train_dir,test_dir,transform=data_transform_aug,transform2=data_transform_simple):
  train_dataset=customdataset(train_dir,data_transform_aug)
  test_dataset=customdataset(test_dir,data_transform_simple)

  class_num=len(train_dataset.classes)

  train_loader_simple=DataLoader(dataset=train_dataset,batch_size=batchs,shuffle=True,num_workers=os.cpu_count())

  test_loader_simple=DataLoader(dataset=test_dataset,batch_size=batchs,shuffle=False,num_workers=os.cpu_count())

  return train_loader_simple,test_loader_simple,len(train_dataset.classes),train_dataset.classes












Writing out_model_set/data_creator.py


In [4]:
%%writefile out_model_set/torch_customer.py




import torch
from torch import nn
import math

device= "cuda" if torch.cuda.is_available() else "cpu"


class LossL1(nn.Module):
  def __init__(self):
    super().__init__()
  def forward(self,predict,targets):
    return torch.sum(torch.abs(predict - targets))/len(targets)

def L1_loss(input, target):
    return (target/2).abs().mean()


def customer_optimizer(model,learning_rate):
    with torch.no_grad():
      for p in model.parameters():
        p -= p.grad *(learning_rate)
      model.zero_grad()


class Binary_cross(nn.Module):
  def __init__(self):
      super().__init__()
  def forward(self,pred,targets):
    #instead: using pred=torch.sigmoid(pred)
    pred=(1 + torch.exp(-(pred))).reciprocal()
    #build up binary cross-entropy
    return torch.mean(-((targets)*torch.log(pred)+(1-targets)*torch.log(1-pred)))

class cross_entropy(nn.Module):
  def __init__(self):
      super().__init__()

  def one_hot(self,x,y):
    return torch.nn.functional.one_hot(x,y)

  def softmax_1(self,x):
    return (1/torch.sum(torch.exp(x),axis=-1)).unsqueeze(axis=1)*torch.exp(x)
   #define the softmax function

  def forward(self,pred,targets):
    # return the lost function based on the probablity "pred" (logits) and y_true targets
    targets=self.one_hot(targets.type(torch.long),pred.shape[-1])
    pred=self.softmax_1(pred)
    sum_one_instance=-1*torch.sum((targets)*torch.log(pred),axis=-1)

    return torch.mean(sum_one_instance)


def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc




class ReLU1(nn.Module):
  def __init__(self):
      super().__init__()
  def forward(self,x):
    return torch.max(torch.tensor([0]).to(device),x)


def Sigmoid(x):
  #build up the sigmoid function
  return (1 + torch.exp(-(x))).reciprocal()

class linear1(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.weight = torch.nn.Parameter(torch.Tensor(out_features, in_features))

        self.bias = torch.nn.Parameter(torch.Tensor(out_features))

        torch.nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))

        fan_in, _ = torch.nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        torch.nn.init.uniform_(self.bias, -bound, bound)



    def forward(self, input):

        return input@self.weight.t()+self.bias


Writing out_model_set/torch_customer.py


In [5]:
%%writefile /content/out_model_set/model_creator.py
import torch_customer
from torch import nn
class tinygvv(nn.Module):
  def __init__(self,in_fea,out_fea,hidden1,hidden2,hidden3,hidden4,hidden5,hidden6):
    super().__init__()

    self.layer1=nn.Sequential(nn.Conv2d(in_channels=in_fea,out_channels=hidden1,kernel_size=3,stride=1,padding=0),
                              torch_customer.ReLU1(),nn.Conv2d(in_channels=hidden1,out_channels=hidden2,kernel_size=3,stride=1,padding=0),torch_customer.ReLU1(),nn.MaxPool2d(2))

    self.layer2=nn.Sequential(nn.Conv2d(in_channels=hidden2,out_channels=hidden3,kernel_size=3,stride=1,padding=0),
                              torch_customer.ReLU1(),nn.Conv2d(in_channels=hidden3,out_channels=hidden4,kernel_size=3,stride=1,padding=0),torch_customer.ReLU1(),nn.MaxPool2d(2))

    self.layer3=nn.Sequential(nn.Conv2d(in_channels=hidden4,out_channels=hidden5,kernel_size=3,stride=1,padding=0),
                              torch_customer.ReLU1(),nn.Conv2d(in_channels=hidden5,out_channels=hidden6,kernel_size=3,stride=1,padding=0),torch_customer.ReLU1(),nn.MaxPool2d(2))

    self.layer4=nn.Sequential(nn.Flatten(),torch_customer.linear1(5000,out_fea)) #turn into feature vetcors size

  def forward(self,x):
    return self.layer4(self.layer3(self.layer2(self.layer1(x))))   #not go back to computation not in one hint



Writing /content/out_model_set/model_creator.py


In [6]:
%%writefile /content/out_model_set/train_creator.py
#train_loader=DataLoader(dataset=train_dataset,batch_size=32,num_workers=2,shuffle=True)
#test_loader=DataLoader(dataset=test_data,batch_size=32,num_workers=2,shuffle=False)

import requests
import zipfile
from pathlib import Path
import pathlib
from PIL import Image
import random
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms


from torch.utils.data import Dataset, DataLoader
import os

device= "cuda" if torch.cuda.is_available() else "cpu"

def train_step(model,data_loader,lost_fn,optimizer,device=device):
  model.train()
  loss=0
  acc=0
  for batch,(X,y) in enumerate(data_loader):
    X=X.to(device)
    y=y.to(device)
    y_logit=model(X)

    y_label=torch.softmax(y_logit,-1).argmax(-1)

    loss_ff=lost_fn(y_logit,y)

    loss=loss+loss_ff.item()

    acc=acc+torch.sum(y_label==y).item()/len(y)


    optimizer.zero_grad()

    loss_ff.backward()

    optimizer.step()

  loss=loss/len(data_loader)

  acc=acc/len(data_loader)

  return loss,acc

def test_step(model,data_loader,lost_fn,device=device):
  model.eval()
  loss=0
  acc=0
  with torch.inference_mode():
    for batch,(X,y) in enumerate(data_loader):
      X=X.to(device)
      y=y.to(device)
      y_logit=model(X)

      y_label=torch.softmax(y_logit,-1).argmax(-1)

      loss_ff=lost_fn(y_logit,y)
      loss=loss+loss_ff.item()

      acc=acc+torch.sum(y_label==y).item()/len(y)

    loss=loss/len(data_loader)

    acc=acc/len(data_loader)

  return loss,acc



def model_fit(model,epochs,train_loaders,test_loaders,loss_fun,optimizer,device=device):

  training_loss=[]
  training_acc=[]
  testing_loss=[]
  testing_acc=[]

  for i in range(epochs):
    a,b=train_step(model,train_loaders,loss_fun,optimizer,device=device)


    c,d=test_step(model,test_loaders,loss_fun,device=device)
    training_loss.append(a)
    training_acc.append(b)
    testing_loss.append(c)
    testing_acc.append(d)
    print(f"training_loss : {a}, training_acc : {b},testing_loss : {c}, testing_acc : {d}")

  return {"training_loss": training_loss,"training acc": training_acc,"testing_loss": testing_loss,"testing acc": testing_acc}





Writing /content/out_model_set/train_creator.py


In [7]:
%%writefile out_model_set/final_train.py
#train_loader=DataLoader(dataset=train_dataset,batch_size=32,num_workers=2,shuffle=True)
#test_loader=DataLoader(dataset=test_data,batch_size=32,num_workers=2,shuffle=False)

import requests
import zipfile
from pathlib import Path
import pathlib
from PIL import Image
import torch_customer
import data_creator,model_creator,train_creator
import matplotlib.pyplot as plt
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
from torch import nn
import argparse
parser = argparse.ArgumentParser(description="This is example of argparse!")
parser.add_argument('--BATCH_size',default=32,type=int)
parser.add_argument("--hidden_unit1",default=256,type=int)
parser.add_argument("--hidden_unit2",default=128,type=int)
parser.add_argument("--hidden_unit3",default=64,type=int)
parser.add_argument("--hidden_unit4",default=32,type=int)
parser.add_argument("--hidden_unit5",default=16,type=int)

parser.add_argument('--NUM_EPOCHS',default=5,type=int)
parser.add_argument('--Learning_rate',default=0.001,type=float)
parser.add_argument("--imagepath",default="/content/Brain-Tumor-Classification-DataSet")
args = parser.parse_args()

imagepath=args.imagepath
hidden_unit1=args.hidden_unit1
hidden_unit2=args.hidden_unit2
hidden_unit3=args.hidden_unit3
hidden_unit4=args.hidden_unit4
hidden_unit5=args.hidden_unit5
batch=args.BATCH_size
NUM_EPOCHS =args.NUM_EPOCHS
Learning_rate=args.Learning_rate

imagepath=Path(imagepath)

train_dir=imagepath/"Training"
test_dir=imagepath/"Testing"
device= "cuda" if torch.cuda.is_available() else "cpu"

train_loader_simple,test_loader_simple,class_num,class_name=data_creator.customdataset_loader(batch,train_dir,test_dir)

loss_fn=torch_customer.cross_entropy()


model_0=model_creator.tinygvv(3,class_num,hidden_unit1,hidden_unit2,hidden_unit3,hidden_unit4,hidden_unit5,8).to(device)


optimizer_0=torch.optim.Adam(model_0.parameters(), lr=Learning_rate)

model_fit_1=train_creator.model_fit(model_0,NUM_EPOCHS,train_loader_simple,test_loader_simple,loss_fn,optimizer_0)


from pathlib import Path

# Create models directory (if it doesn't already exist), see: https://docs.python.org/3/library/pathlib.html#pathlib.Path.mkdir
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, # create parent directories if needed
                 exist_ok=True # if models directory already exists, don't error
)

# Create model save path
MODEL_NAME = "MODEL_PATH222"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME


torch.save(model_0.state_dict(),f=MODEL_SAVE_PATH)


Writing out_model_set/final_train.py


In [8]:
%%writefile open_test_2.py

import requests
request=requests.get('https://cdnintech.com/media/chapter/43882/1512345123/media/image3.png')
#https://www.intechopen.com/chapters/43882
#request=requests.get('https://cdn.the-scientist.com/assets/articleNo/70327/iImg/47139/glioblastoma-inline-l.webp')
with open("/content/Brain-Tumor-Classification-DataSet/test_image2", "wb") as f:
  f.write(request.content)
#import requests
#import Image
#Image.open(MODEL_PATH/"test_image2")




Writing open_test_2.py


In [9]:
%%writefile out_model_set/final_pred.py


from torchvision import  transforms
import torchvision
from PIL import Image
from pathlib import Path
import torch
import model_creator,data_creator

import argparse
parser = argparse.ArgumentParser(description="This is example of argparse!")
parser.add_argument('--imagepath',default="test_image2")
parser.add_argument("--hidden_unit1",default=256,type=int)
parser.add_argument("--hidden_unit2",default=128,type=int)
parser.add_argument("--hidden_unit3",default=64,type=int)
parser.add_argument("--hidden_unit4",default=32,type=int)
parser.add_argument("--hidden_unit5",default=16,type=int)

parser.add_argument('--NUM_EPOCHS',default=5,type=int)
parser.add_argument('--Learning_rate',default=0.001,type=float)

args = parser.parse_args()

imagepath=args.imagepath
hidden_unit1=args.hidden_unit1
hidden_unit2=args.hidden_unit2
hidden_unit3=args.hidden_unit3
hidden_unit4=args.hidden_unit4
hidden_unit5=args.hidden_unit5

def convertion_function(path0):

  device= "cuda" if torch.cuda.is_available() else "cpu"
  newImagePath=Path("/content/Brain-Tumor-Classification-DataSet")

  custom_uni8=torchvision.io.read_image(newImagePath/path0)
  custom_uni8=custom_uni8.type(torch.float).to(device)/255

  return custom_uni8

def custom_loader(path0=imagepath):

  data_transform_simple=transforms.Compose(
    [transforms.Resize((228,228))])

  image=data_transform_simple(convertion_function(path0))
  image=image.unsqueeze(0)


  device= "cuda" if torch.cuda.is_available() else "cpu"





  load_path=Path("/content/models/MODEL_PATH222")


  load_model=model_creator.tinygvv(3,4,hidden_unit1,hidden_unit2,hidden_unit3,hidden_unit4,hidden_unit5,8).to(device)


  load_model.load_state_dict(torch.load(f=load_path,map_location=torch.device('cpu')))

  load_model.eval()

  with torch.inference_mode():
    y_logit=load_model(image)

    y_label=torch.softmax(y_logit,-1).argmax(-1)

  return y_label.item()


if __name__ == "__main__":
  print (custom_loader())



Writing out_model_set/final_pred.py


In [10]:
! git clone https://github.com/SartajBhuvaji/Brain-Tumor-Classification-DataSet.git

Cloning into 'Brain-Tumor-Classification-DataSet'...
remote: Enumerating objects: 3039, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 3039 (delta 0), reused 0 (delta 0), pack-reused 3035
Receiving objects: 100% (3039/3039), 79.25 MiB | 13.56 MiB/s, done.
Updating files: 100% (3264/3264), done.


In [11]:
!python open_test_2.py

In [12]:
!rm open_test_2.py

In [13]:
!rm path_loader.py

In [20]:
!python out_model_set/final_train.py --NUM_EPOCHS 10 --hidden_unit1 256 --hidden_unit2  256 --hidden_unit3 128 --hidden_unit4 128  --hidden_unit5 16

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 7.15 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
training_loss : 1.2565650039248997, training_acc : 0.41622474747474747,testing_loss : 2.051750829586616, testing_acc : 0.26778846153846153
training_loss : 1.0105082445674471, training_acc : 0.561584595959596,testing_loss : 1.747493354173807, testing_acc : 0.3591346153846154
training_loss : 0.9027382029427422, training_acc : 0.6277146464646465,testing_loss : 1.5375316716157472, testing_acc : 0.4225961538461539
training_loss : 0.8801874028311836, training_acc : 0.6402146464646464,testing_loss : 1.3655330194876745, testing_acc : 0.5139423076923078
training_loss : 0.8088463313049741, training_acc : 0.6706123737373737,testing_loss : 1.503236974661

In [15]:
!python out_model_set/final_pred.py  --hidden_unit1 128 --hidden_unit2  64 --hidden_unit3 32 --hidden_unit4 32  --hidden_unit5 16

1


In [16]:
!python out_model_set/final_pred.py  --imagepath "Testing/pituitary_tumor/image(54).jpg" --hidden_unit1 128 --hidden_unit2  64 --hidden_unit3 32 --hidden_unit4 32  --hidden_unit5 16

3


In [17]:
import torch

import torchvision

print (torchvision.__version__,torch.__version__)

0.18.0+cu121 2.3.0+cu121


In [18]:
#from out_model_set import data_creator,model_creator,train_creator



In [19]:
#data_creator.customdataset_loader()